In [1]:
! git clone https://github.com/babaknaderi/TextComplexityDE.git

Cloning into 'TextComplexityDE'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 21 (delta 7), reused 16 (delta 4), pack-reused 0
Unpacking objects: 100% (21/21), 392.71 KiB | 5.69 MiB/s, done.


In [2]:
import pandas as pd

# Prepare Data

In [3]:
ratings = pd.read_csv("/kaggle/working/TextComplexityDE/TextComplexityDE19/ratings.csv", encoding="latin-1")
parallel = pd.read_csv("/kaggle/working/TextComplexityDE/TextComplexityDE19/parallel_corpus.csv", encoding="latin-1")
source = pd.read_csv("/kaggle/working/TextComplexityDE/TextComplexityDE19/source.csv", encoding="latin-1")

In [4]:
parallel.head(3)

,Sentence_Id,Article_ID,Article,Original_Sentence,Simplification,Rating
0,5,1,Seifenblase,"Wegen dieser leichten Vergänglichkeit wurde ,S...","Weil Seifenblasen nicht lange halten, wurden s...",Etwas einfacher
1,7,1,Seifenblase,In der Kunst wird spätestens seit dem Barock d...,In der Kunst wird die Seifenblase spätestens s...,Deutlich einfacher
2,11,1,Seifenblase,"Eine Seifenblase entsteht, wenn sich ein dünne...","Eine Seifenblase entsteht, wenn sich eine klei...",Etwas einfacher


In [5]:
! pip install textstat -q

In [6]:
import textstat

test_data = (
    "Playing games has always been thought to be important to "
    "the development of well-balanced and creative children; "
    "however, what part, if any, they should play in the lives "
    "of adults has never been researched that deeply. I believe "
    "that playing games is every bit as important for adults "
    "as for children. Not only is taking time out to play games "
    "with our children and other adults valuable to building "
    "interpersonal relationships but is also a wonderful way "
    "to release built up tension."
)

metrics = {
    "flesch_reading_ease":textstat.flesch_reading_ease,
    "flesch_kincaid_grade":textstat.flesch_kincaid_grade,
    "smog_index":textstat.smog_index,
    "coleman_liau_index":textstat.coleman_liau_index,
    "automated_readability_index":textstat.automated_readability_index,
    "dale_chall_readability_score":textstat.dale_chall_readability_score,
    "difficult_words":textstat.difficult_words,
    "linsear_write_formula":textstat.linsear_write_formula,
    "gunning_fog":textstat.gunning_fog,
    # "text_standard":textstat.text_standard,
    "fernandez_huerta":textstat.fernandez_huerta,
    "szigriszt_pazos":textstat.szigriszt_pazos,
    "gutierrez_polini":textstat.gutierrez_polini,
    "crawford":textstat.crawford,
    "gulpease_index":textstat.gulpease_index,
    "osman":textstat.osman,
}

for key in metrics.keys():
    print(key, metrics[key](test_data))

flesch_reading_ease 52.23
flesch_kincaid_grade 12.8
smog_index 12.5
coleman_liau_index 11.03
automated_readability_index 15.5
dale_chall_readability_score 7.3
difficult_words 9
linsear_write_formula 16.333333333333332
gunning_fog 12.38
fernandez_huerta 88.99
szigriszt_pazos 87.57
gutierrez_polini 38.79
crawford 3.1
gulpease_index 50.7
osman 48.01


In [7]:
results = []

for sentence in parallel.Original_Sentence:
    result_part = []
    for key in metrics.keys():
        result_part.append(metrics[key](sentence))
    results.append(result_part)

df1 = pd.DataFrame(results)

results = []

for sentence in parallel.Simplification:
    result_part = []
    for key in metrics.keys():
        result_part.append(metrics[key](sentence))
    results.append(result_part)

df2 = pd.DataFrame(results)

In [8]:
parallel.Rating.value_counts()

Rating
Etwas einfacher                                      135
Deutlich einfacher                                   114
Nicht einfacher / konnte nicht vereinfacht werden      1
Name: count, dtype: int64

In [9]:
df = df1 - df2

rating_map = {
    "Nicht einfacher / konnte nicht vereinfacht werden":0,
    "Etwas einfacher":1,
    "Deutlich einfacher":2,
}

df["rating"] = parallel.Rating.map(rating_map)

In [10]:
df = df[df.rating!=0]

# Train Logistic Regression Model

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming your DataFrame is named 'df'
# Separate features and target variable
X = df.drop('rating', axis=1)
y = df['rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

# Now, you can use the trained model to predict ratings for new data
# For example, if you have a new DataFrame 'new_data' with similar columns:
# new_predictions = model.predict(new_data)

Accuracy: 0.52
Confusion Matrix:
[[20 10]
 [14  6]]
Classification Report:
              precision    recall  f1-score   support

           1       0.59      0.67      0.62        30
           2       0.38      0.30      0.33        20

    accuracy                           0.52        50
   macro avg       0.48      0.48      0.48        50
weighted avg       0.50      0.52      0.51        50



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Inference

In [12]:
example = [
    """Während der aktuellen COVID-19-Pandemie ist die schnelle Verfügbarkeit fundierter Informationen 
    von entscheidender Bedeutung, um Informationen über Diagnose, Krankheitsverlauf, Behandlung abzuleiten 
    oder die Verhaltensregeln in der Öffentlichkeit anzupassen.""",
    """Während der aktuellen COVID-19-Pandemie ist es wichtig, dass wir schnell an fundierte Informationen 
    gelangen können. Diese Informationen sind entscheidend, um mehr über die Diagnose, den Krankheitsverlauf
    und die Behandlung zu erfahren. Außerdem helfen sie dabei, die Verhaltensregeln 
    in der Öffentlichkeit anzupassen."""
]

def predict_easyfication_rating(example):
    result_part0 = []
    for key in metrics.keys():
        result_part0.append(metrics[key](example[0]))

    result_part1 = []
    for key in metrics.keys():
        result_part1.append(metrics[key](example[1]))

    result_df = pd.DataFrame(result_part0) - pd.DataFrame(result_part1)

    return model.predict(result_df.T)

predict_easyfication_rating(example)

array([2])

In [13]:
predict_easyfication_rating([example[0], example[0]])

array([1])

# Train Random Forest Model

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming your DataFrame is named 'df'
# Separate features and target variable
X = df.drop('rating', axis=1)
y = df['rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

# Now, you can use the trained Random Forest model to predict ratings for new data
# For example, if you have a new DataFrame 'new_data' with similar columns:
# new_predictions = model.predict(new_data)

Accuracy: 0.5
Confusion Matrix:
[[21  9]
 [16  4]]
Classification Report:
              precision    recall  f1-score   support

           1       0.57      0.70      0.63        30
           2       0.31      0.20      0.24        20

    accuracy                           0.50        50
   macro avg       0.44      0.45      0.43        50
weighted avg       0.46      0.50      0.47        50



# Train Naive Bayes Model

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming your DataFrame is named 'df'
# Separate features and target variable
X = df.drop('rating', axis=1)
y = df['rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Gaussian Naive Bayes classifier
model = GaussianNB()

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

# Now, you can use the trained Naive Bayes model to predict ratings for new data
# For example, if you have a new DataFrame 'new_data' with similar columns:
# new_predictions = model.predict(new_data)

Accuracy: 0.62
Confusion Matrix:
[[24  6]
 [13  7]]
Classification Report:
              precision    recall  f1-score   support

           1       0.65      0.80      0.72        30
           2       0.54      0.35      0.42        20

    accuracy                           0.62        50
   macro avg       0.59      0.57      0.57        50
weighted avg       0.60      0.62      0.60        50

